In [1]:
import os

In [2]:
%pwd

'd:\\ML\\spelling correction\\Spelling-correction-project\\resreach'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\ML\\spelling correction\\Spelling-correction-project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [6]:
from SpellingCorrection.constants import*
from SpellingCorrection.utils.common import read_yaml, create_directories

In [8]:
import os
import yaml
from dataclasses import dataclass 
from pathlib import Path


class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self)->DataTransformationConfig:
            config =self.config.data_transformation
            create_directories([config.root_dir])

            data_transformation_config= DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
            )
            return data_transformation_config

In [9]:
import os
import torch

from SpellingCorrection.logging import logger
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer
from tqdm.notebook import tqdm
from datasets import load_from_disk

[2024-06-10 13:34:28,074 : INFO : config : PyTorch version 2.3.0 available.]


In [19]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
    
    def convert_examples_to_features(self, example_batch):
        # Làm sạch dữ liệu để loại bỏ các giá trị không hợp lệ
        example_batch['noisy_phrases'] = [str(x) if x is not None else "" for x in example_batch['noisy_phrases']]
        example_batch['sentence']= [str(x) if x is not None else "" for x in example_batch['sentence']]
        input_encodings=self.tokenizer(example_batch['noisy_phrases'], max_length=128,padding="max_length", truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['sentence'],max_length=128, padding="max_length", truncation=True)

        return {
            "input_ids": input_encodings['input_ids'],
            "attention_mask": input_encodings['attention_mask'],
            "labels": target_encodings['input_ids'],
        }
    def convert(self):
        
        dataset=load_from_disk(self.config.data_path)
        dataset_pt=dataset.map(self.convert_examples_to_features, batched=True)
        dataset_pt.save_to_disk(os.path.join(self.config.root_dir, "dataset"))

In [20]:
import pandas as pd
train_df = pd.read_csv('D:\\ML\\spelling correction\\Spelling-correction-project\\artifacts\\data_ingestion\\dataset\\train_data.csv', encoding='ISO-8859-1' ,on_bad_lines='skip')
test_df = pd.read_csv('D:\\ML\\spelling correction\\Spelling-correction-project\\artifacts\\data_ingestion\\dataset\\test_data.csv', encoding='ISO-8859-1', on_bad_lines='skip')
val_df = pd.read_csv('D:\\ML\\spelling correction\\Spelling-correction-project\\artifacts\\data_ingestion\\dataset\\eval_data.csv', encoding='ISO-8859-1', on_bad_lines='skip')


print("Train Data Columns:", train_df.columns)
print("Validation Data Columns:", val_df.columns)
print("Test Data Columns:", test_df.columns)

Train Data Columns: Index(['sentence', 'noisy_phrases'], dtype='object')
Validation Data Columns: Index(['sentence', 'noisy_phrases'], dtype='object')
Test Data Columns: Index(['sentence', 'noisy_phrases'], dtype='object')


In [21]:
try:
    config= ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation=DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-06-10 13:45:29,857 : INFO : common : Reading the yaml file from the path: config\config.yaml loaded successfully]
[2024-06-10 13:45:29,858 : INFO : common : Reading the yaml file from the path: params.yaml loaded successfully]


c:\ProgramData\miniconda3\envs\spelling\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/37616 [00:00<?, ? examples/s]

c:\ProgramData\miniconda3\envs\spelling\Lib\site-packages\transformers\tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/9404 [00:00<?, ? examples/s]

Map:   0%|          | 0/11755 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/37616 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9404 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/11755 [00:00<?, ? examples/s]